# Import Lib

In [ ]:
import tensorflow as tf
from PIL import ImageFile
import pandas as pd
from tqdm import tqdm
import numpy as np
import os
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess_input 
from tensorflow.keras.models import load_model

# Submission of models to meet the imagenet supported standards

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

# --- Configuration Constants ---
IMAGE_SIZE_TUPLE = (224, 224)
IMG_HEIGHT, IMG_WIDTH = IMAGE_SIZE_TUPLE
IMG_CHANNELS = 3
BATCH_SIZE = 32

idx_to_ilsvrc_id_df = pd.read_csv(
    "https://github.com/TheoBourdais/ImageNetSubmission/raw/main/src/idx_to_ILSVRC_ID.csv"
)
idx_to_ilsvrc_id = {
    idx: ilsvrc_id
    for idx, ilsvrc_id in zip(idx_to_ilsvrc_id_df["idx"], idx_to_ilsvrc_id_df["ILSVRC_ID"])
}

# --- ปรับปรุง Preprocessing function ---
def preprocess_image_tf(image_path, target_height=IMG_HEIGHT, target_width=IMG_WIDTH):
    img_bytes = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS) 
    img = tf.image.resize(img, [target_height, target_width])  
    img = resnet_preprocess_input(img) # <--- ใช้ ResNet50 specific preprocessing
    return image_path, img

def create_tf_dataset(image_folder_path, batch_size):
    image_paths_pattern = os.path.join(image_folder_path, "**", "*.JPEG")
    print(f"Searching for images with pattern: {image_paths_pattern}")
    image_paths = sorted(tf.io.gfile.glob(image_paths_pattern))

    if not image_paths:
        image_paths_pattern_lower = os.path.join(image_folder_path, "**", "*.jpeg")
        print(f"No *.JPEG found, trying pattern: {image_paths_pattern_lower}")
        image_paths = sorted(tf.io.gfile.glob(image_paths_pattern_lower))
        if not image_paths:
            image_paths_pattern_jpg = os.path.join(image_folder_path, "**", "*.jpg")
            print(f"No *.jpeg found, trying pattern: {image_paths_pattern_jpg}")
            image_paths = sorted(tf.io.gfile.glob(image_paths_pattern_jpg))
            if not image_paths:
                raise ValueError(f"No *.JPEG, *.jpeg, or *.jpg images found recursively in: {image_folder_path}")

    print(f"Found {len(image_paths)} images.")
    filename_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    image_dataset = filename_dataset.map(
        lambda x: preprocess_image_tf(x, IMG_HEIGHT, IMG_WIDTH),
        num_parallel_calls=tf.data.AUTOTUNE
    )
    batched_dataset = image_dataset.batch(batch_size)
    return batched_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

def get_test_submission_tf(model, tf_dataloader):
    all_filenames = []
    all_predictions_np = []
    print("Processing images for submission...")
    for batch_paths_tensor, batch_images_tensor in tqdm(tf_dataloader):
        outputs = model(batch_images_tensor, training=False) # หรือ model.predict(batch_images_tensor)
        _, predicted_indices = tf.math.top_k(outputs, k=5)
        predicted_indices_np = predicted_indices.numpy()
        batch_paths_str = [os.path.basename(p.numpy().decode('utf-8')) for p in batch_paths_tensor]
        all_filenames.extend(batch_paths_str)
        all_predictions_np.extend(predicted_indices_np)
    mapped_predictions = []
    for pred_indices_row in all_predictions_np:
        mapped_predictions.append([idx_to_ilsvrc_id[idx_val] for idx_val in pred_indices_row])
    submission_df = pd.DataFrame(
        mapped_predictions,
        columns=[f"choice {i}" for i in range(1, 6)],
        index=all_filenames
    )
    submission_df = submission_df.sort_index()
    return submission_df

if __name__ == '__main__':
    print(f"TensorFlow Version: {tf.__version__}")

    keras_model_file = "/media/capybara/Data/dataset_vit/resnet50_phase2_explicit_save.keras" 

    print(f"Loading Keras model from: {keras_model_file}")
    try:
        model = load_model(keras_model_file) # <--- ใช้ load_model
        print("Keras model loaded successfully.")
    except Exception as e:
        print(f"Error loading Keras model: {e}")
        exit()
    model.summary()

    validation_set_path = "/media/capybara/Data/dataset_vit/testdata/" # <--- ตรวจสอบ path นี้
    print(f"Preparing TensorFlow dataset from: {validation_set_path}")
    try:
        test_tf_dataset = create_tf_dataset(validation_set_path, batch_size=BATCH_SIZE)
    except ValueError as e:
        print(f"Error creating dataset: {e}")
        print("Please check the 'validation_set_path' and ensure it contains JPEG/jpeg/jpg images in subdirectories.")
        exit()

    print("Generating submission file...")
    submission_df = get_test_submission_tf(model, test_tf_dataset)

    submission_file_path = "submission_tf_resnet50.txt"
    print(f"Saving submission to: {submission_file_path}")
    submission_df.to_csv(submission_file_path, index=False, header=False, sep=" ")
    print(f"Submission file '{submission_file_path}' created successfully.")
    print("Done.")

TensorFlow Version: 2.19.0
Loading Keras model from: /media/capybara/Data/dataset_vit/resnet50_phase2_explicit_save.keras


I0000 00:00:1748246318.594310   15722 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7299 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:04:00.0, compute capability: 6.1


Keras model loaded successfully.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

...

 Non-trainable params: 8,611,712 (32.85 MB)

 Optimizer params: 36,198,354 (138.09 MB)

Preparing TensorFlow dataset from: /media/capybara/Data/dataset_vit/testdata/
Searching for images with pattern: /media/capybara/Data/dataset_vit/testdata/**/*.JPEG
Found 100000 images.
Generating submission file...
Processing images for submission...


100%|██████████| 3125/3125 [22:09<00:00,  2.35it/s]


Saving submission to: submission_tf_resnet50.txt
Submission file 'submission_tf_resnet50.txt' created successfully.
Done.
